In [1]:
import geopandas as gpd


POI_path = "raw_data/POI\SG-POI.shp"
Train_path = "raw_data/土地利用\SG-Landuse-训练集.shp"
BUSWAY_path = "raw_data/路网\SG-公路线.shp"
SUBWAY_path = "raw_data/路网\SG-铁路线.shp"
SUBWAY_STATION_path = "raw_data/交通设施\SG_交通设施-railway_station.shp"
BUS_STATION_path = "raw_data/交通设施\SG_交通设施-bus_station.shp"
CROSSING_path = "raw_data/交通设施\SG_交通设施-crossing.shp"
POI_gdf = gpd.read_file(POI_path)
Train_gdf = gpd.read_file(Train_path)
BUSWAY_gdf = gpd.read_file(BUSWAY_path)
SUBWAY_gdf = gpd.read_file(SUBWAY_path)
SUBWAY_STATION_gdf = gpd.read_file(SUBWAY_STATION_path)
BUS_STATION_gdf = gpd.read_file(BUS_STATION_path)
CROSSING_gdf = gpd.read_file(CROSSING_path)

In [2]:
# 执行空间连接
    # 检查坐标参考系统(CRS)
if Train_gdf.crs is None:
    print("警告: 数据没有定义坐标参考系统.")
else:
    print(f"坐标参考系统: {Train_gdf.crs}")

Train_gdf = Train_gdf.to_crs(epsg=4326)
print(Train_gdf)

坐标参考系统: PROJCS["SVY21",GEOGCS["GCS_SVY21_WGS84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",1.36666666666667],PARAMETER["central_meridian",103.833333333333],PARAMETER["scale_factor",1],PARAMETER["false_easting",28001.642],PARAMETER["false_northing",38744.572],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
        OBJECTID_1  OBJECTID                  LU_DESC LU_TEXT   GPR  WHI_Q_MX  \
0                1         1             RESERVE SITE    None   EVA       0.0   
1                2         2                     ROAD    None  None       0.0   
2                3         3              RESIDENTIAL    None   1.4       0.0   
3                4         4  EDUCATIONAL INSTITUTION       E   EVA       0.0   
4                5         5              RESIDENTIAL    None 

In [3]:
#清洗数据
columns_to_drop = ['LU_TEXT', 'WHI_Q_MX', 'GPR_B_MN', 'INC_CRC', 'X_ADDR', 'Y_ADDR', '类目', 'Shape_Le_1', 'OBJECTID']
Train_gdf.drop(columns=columns_to_drop, inplace=True)
print("剩余的列：", Train_gdf.columns.tolist())

剩余的列： ['OBJECTID_1', 'LU_DESC', 'GPR', 'SHAPE_Leng', 'Shape_Area', 'YDR', 'geometry']


In [4]:
print(SUBWAY_STATION_gdf)
print(BUS_STATION_gdf)

     OBJECTID       osm_id  code           fclass          name  \
0           1    206477134  5601  railway_station      Aljunied   
1           2    206493462  5601  railway_station       Kallang   
2           3    206529415  5601  railway_station  Yio Chu Kang   
3           4    207725954  5601  railway_station   Tanah Merah   
4           5    207725957  5601  railway_station         Bedok   
..        ...          ...   ...              ...           ...   
105      5515   6589739733  5601  railway_station         Beach   
106      5582   7678872567  5601  railway_station    Ang Mo Kio   
107      5825   9160399644  5601  railway_station         Elias   
108      5832   9168360797  5601  railway_station   Bright Hill   
109      5952  10538809251  5601  railway_station  HarbourFront   

                      geometry  
0    POINT (103.88291 1.31645)  
1    POINT (103.87134 1.31147)  
2    POINT (103.84494 1.38179)  
3     POINT (103.9465 1.32726)  
4    POINT (103.93022 1.32398)

In [5]:
# 执行空间连接：公交站点
connected_bus_station_gdf = gpd.sjoin(Train_gdf, BUS_STATION_gdf, how="left", predicate='contains')
filtered_bus_stations_for_count = connected_bus_station_gdf.dropna(subset=['osm_id'])
BUS_STATION_counts = filtered_bus_stations_for_count.groupby('OBJECTID_1').size().reset_index(name='bus_station_counts')
final_gdf = Train_gdf.merge(BUS_STATION_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')
final_gdf['bus_station_counts'] = final_gdf['bus_station_counts'].fillna(0).astype(int)


# 统计地铁站数量
connected_subway_station_gdf = gpd.sjoin(final_gdf, SUBWAY_STATION_gdf, how="left", predicate='contains')
filtered_subway_stations_for_count = connected_subway_station_gdf.dropna(subset=['osm_id'])
SUBWAY_STATION_counts = filtered_subway_stations_for_count.groupby('OBJECTID_1').size().reset_index(name='subway_station_counts')
final_gdf = final_gdf.merge(SUBWAY_STATION_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')
final_gdf['subway_station_counts'] = final_gdf['subway_station_counts'].fillna(0).astype(int)

# 统计地铁站数量
connected_crossing_gdf = gpd.sjoin(final_gdf, CROSSING_gdf, how="left", predicate='contains')
filtered_crossing_for_count = connected_crossing_gdf.dropna(subset=['osm_id'])
CROSSING_counts = filtered_crossing_for_count.groupby('OBJECTID_1').size().reset_index(name='crossing_counts')
final_gdf = final_gdf.merge(CROSSING_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')
final_gdf['crossing_counts'] = final_gdf['crossing_counts'].fillna(0).astype(int)

print(final_gdf)

        OBJECTID_1                  LU_DESC   GPR   SHAPE_Leng    Shape_Area  \
0                1             RESERVE SITE   EVA  1491.621125  55268.807809   
1                2                     ROAD  None  4468.834966  61886.993286   
2                3              RESIDENTIAL   1.4  1573.440329  42320.207628   
3                4  EDUCATIONAL INSTITUTION   EVA   864.088998  40964.380883   
4                5              RESIDENTIAL   3.2   994.619480  63201.884130   
...            ...                      ...   ...          ...           ...   
110798      110799                     None  None   367.444406   2127.078820   
110799      110800                     ROAD  None   889.130470   4775.703468   
110800      110801                     None  None   124.560204    453.470433   
110801      110802                     ROAD  None   841.963415  10757.633805   
110802      110803              RESIDENTIAL   2.8   126.337295    946.217982   

        YDR                            

In [6]:
unique_fclasses = POI_gdf['fclass'].unique()
print(unique_fclasses)


['bakery' 'artwork' 'toilet' 'attraction' 'fast_food' 'convenience' 'cafe'
 'supermarket' 'food_court' 'bank' 'post_office' 'restaurant' 'monument'
 'shelter' 'tourist_info' 'viewpoint' 'bench' 'atm' 'mall' 'college'
 'telephone' 'memorial' 'bar' 'pub' 'pharmacy' 'theatre'
 'department_store' 'school' 'playground' 'water_tower' 'cinema'
 'observation_tower' 'fire_station' 'vending_any' 'dentist' 'post_box'
 'stadium' 'hospital' 'police' 'recycling' 'museum' 'camera_surveillance'
 'library' 'car_dealership' 'laundry' 'lighthouse' 'sports_centre'
 'fountain' 'gift_shop' 'doityourself' 'toy_shop' 'florist' 'hairdresser'
 'clothes' 'beverages' 'jeweller' 'vending_machine' 'kindergarten'
 'greengrocer' 'butcher' 'waste_basket' 'community_centre'
 'drinking_water' 'clinic' 'market_place' 'general' 'picnic_site'
 'bookshop' 'travel_agent' 'camp_site' 'embassy' 'town_hall' 'doctors'
 'bicycle_rental' 'zoo' 'theme_park' 'comms_tower' 'tower' 'guesthouse'
 'bicycle_shop' 'nightclub' 'battlefield

In [7]:
categories = {
    '餐饮服务': ['bakery', 'fast_food', 'restaurant', 'cafe', 'food_court', 'bar', 'pub', 'ice_cream', 'biergarten'],
    '购物服务': ['supermarket', 'convenience', 'department_store', 'clothes', 'butcher', 'greengrocer', 'beverages', 'florist', 'chemist', 'doityourself', 'gift_shop', 'jeweller', 'toy_shop', 'bookshop', 'video_shop', 'mobile_phone_shop', 'computer_shop', 'furniture_shop', 'sports_shop', 'bicycle_shop', 'outdoor_shop', 'kiosk', 'newsagent', 'stationery', 'general', 'car_dealership'],
    '公共服务': ['bank', 'atm', 'post_office', 'telephone', 'recycling', 'waste_basket', 'clinic', 'hospital', 'doctors', 'dentist', 'veterinary', 'pharmacy', 'police', 'fire_station', 'town_hall', 'community_centre', 'embassy', 'courthouse', 'prison', 'public_building', 'library', 'theatre', 'arts_centre', 'museum', 'zoo', 'theme_park', 'camp_site', 'nightclub', 'casino', 'laundry', 'car_wash', 'car_rental', 'bicycle_rental', 'car_sharing'],
    '交通设施': ['railway_station', 'bus_station', 'subway_entrance', 'parking', 'fuel', 'charging_station', 'taxi', 'rental_bicycle', 'rental_car'],
    '休闲娱乐': ['cinema', 'theatre', 'arts_centre', 'stadium', 'swimming_pool', 'sports_centre', 'pitch', 'playground', 'dog_park', 'garden_centre', 'park', 'picnic_site', 'attraction', 'viewpoint', 'memorial', 'monument', 'castle', 'ruins', 'fort', 'wayside_cross', 'wayside_shrine', 'observation_tower', 'tower', 'lighthouse', 'water_tower', 'water_well', 'fountain', 'waterfall', 'beach', 'cliff', 'cave_entrance', 'spring', 'tree', 'forest', 'nature_reserve', 'national_park', 'wildlife_hide', 'chalet', 'hostel', 'guesthouse', 'hotel', 'motel', 'camp_site', 'theme_park', 'zoo', 'battlefield']
}

# 根据 fclass 进行分割，并直接创建五个GeoDataFrame
catering_gdf = POI_gdf[POI_gdf['fclass'].isin(categories['餐饮服务'])].copy()
shopping_gdf = POI_gdf[POI_gdf['fclass'].isin(categories['购物服务'])].copy()
public_service_gdf = POI_gdf[POI_gdf['fclass'].isin(categories['公共服务'])].copy()
transportation_gdf = POI_gdf[POI_gdf['fclass'].isin(categories['交通设施'])].copy()
leisure_gdf = POI_gdf[POI_gdf['fclass'].isin(categories['休闲娱乐'])].copy()

In [8]:
print(catering_gdf)

       OBJECTID  code      fclass              name     POINT_X   POINT_Y  \
0             1  2502      bakery      Yio Chu Kang  103.844936  1.381791   
4             5  2302   fast_food        McDonald's  103.887257  1.396785   
6             7  2303        cafe              None  103.886287  1.389365   
10           11  2302   fast_food        McDonald's  103.901461  1.385294   
11           12  2302   fast_food               KFC  103.901416  1.385232   
...         ...   ...         ...               ...         ...       ...   
22543     22544  2306  food_court              None  103.766958  1.313524   
22545     22546  2301  restaurant              None  103.767473  1.317662   
22554     22555  2301  restaurant              None  103.766855  1.315035   
22555     22556  2301  restaurant  Bangladeshi Food  103.855213  1.308597   
22557     22558  2301  restaurant          Al Bismi  103.855232  1.308877   

                        geometry  
0      POINT (103.84494 1.38179)  
4    

In [9]:
# 统计POI
connected_catering_gdf = gpd.sjoin(final_gdf, CROSSING_gdf, how="left", predicate='contains')
filtered_catering_for_count = connected_catering_gdf.dropna(subset=['index_right'])
CATERING_counts = filtered_catering_for_count.groupby('OBJECTID_1').size().reset_index(name='catering')
final_gdf = final_gdf.merge(CATERING_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')

final_gdf['catering'] = final_gdf['catering'].fillna(0).astype(int)
connected_shopping_gdf = gpd.sjoin(final_gdf, shopping_gdf, how="left", predicate='contains')
filtered_shopping_for_count = connected_shopping_gdf.dropna(subset=['index_right'])
SHOPPING_counts = filtered_shopping_for_count.groupby('OBJECTID_1').size().reset_index(name='shopping')
final_gdf = final_gdf.merge(SHOPPING_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')
final_gdf['shopping'] = final_gdf['shopping'].fillna(0).astype(int)

connected_public_service_gdf = gpd.sjoin(final_gdf, public_service_gdf, how="left", predicate='contains')
filtered_public_service_for_count = connected_public_service_gdf.dropna(subset=['index_right'])
SERVICE_counts = filtered_public_service_for_count.groupby('OBJECTID_1').size().reset_index(name='public_service')
final_gdf = final_gdf.merge(SERVICE_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')
final_gdf['public_service'] = final_gdf['public_service'].fillna(0).astype(int)

connected_transportation_gdf = gpd.sjoin(final_gdf, transportation_gdf, how="left", predicate='contains')
filtered_transportation_for_count = connected_transportation_gdf.dropna(subset=['index_right'])
TRANSPORTATION_counts = filtered_transportation_for_count.groupby('OBJECTID_1').size().reset_index(name='transportation')
final_gdf = final_gdf.merge(TRANSPORTATION_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')
final_gdf['transportation'] = final_gdf['transportation'].fillna(0).astype(int)

connected_leisure_gdf = gpd.sjoin(final_gdf, leisure_gdf, how="left", predicate='contains')
filtered_leisure_for_count = connected_leisure_gdf.dropna(subset=['index_right'])
LEISURE_counts = filtered_leisure_for_count.groupby('OBJECTID_1').size().reset_index(name='leisure')
final_gdf = final_gdf.merge(LEISURE_counts, left_on='OBJECTID_1', right_on='OBJECTID_1', how='left')
final_gdf['leisure'] = final_gdf['leisure'].fillna(0).astype(int)
print(final_gdf)
# 定义输出路径
output_path = 'Block_info.csv'

# 保存结果到CSV文件
final_gdf.to_csv(output_path, index=True, encoding='utf-8-sig')

        OBJECTID_1                  LU_DESC   GPR   SHAPE_Leng    Shape_Area  \
0                1             RESERVE SITE   EVA  1491.621125  55268.807809   
1                2                     ROAD  None  4468.834966  61886.993286   
2                3              RESIDENTIAL   1.4  1573.440329  42320.207628   
3                4  EDUCATIONAL INSTITUTION   EVA   864.088998  40964.380883   
4                5              RESIDENTIAL   3.2   994.619480  63201.884130   
...            ...                      ...   ...          ...           ...   
110798      110799                     None  None   367.444406   2127.078820   
110799      110800                     ROAD  None   889.130470   4775.703468   
110800      110801                     None  None   124.560204    453.470433   
110801      110802                     ROAD  None   841.963415  10757.633805   
110802      110803              RESIDENTIAL   2.8   126.337295    946.217982   

        YDR                            

In [10]:
# 路网数据清理
import pandas as pd
non_null_name_gdf = SUBWAY_gdf.dropna(subset=['name'])
null_name_gdf = SUBWAY_gdf[SUBWAY_gdf['name'].isna()]
cleaned_non_null_gdf = non_null_name_gdf.sort_values(by=['name', 'Shape_Leng'], ascending=[True, False]).drop_duplicates(subset='name', keep='first')
cleaned_SUBWAY_gdf = pd.concat([cleaned_non_null_gdf, null_name_gdf], ignore_index=True)
print(cleaned_SUBWAY_gdf)

non_null_name_gdf = BUSWAY_gdf.dropna(subset=['name'])
null_name_gdf = BUSWAY_gdf[BUSWAY_gdf['name'].isna()]
cleaned_non_null_gdf = non_null_name_gdf.sort_values(by=['name', 'Shape_Leng'], ascending=[True, False]).drop_duplicates(subset='name', keep='first')
cleaned_BUSWAY_gdf = pd.concat([cleaned_non_null_gdf, null_name_gdf], ignore_index=True)
print(cleaned_BUSWAY_gdf)

      OBJECTID      osm_id  code    fclass                            name  \
0          304   428907035  6105  monorail               Bukit Panjang LRT   
1          301   428907032  6105  monorail         Bukit Panjang LRT Depot   
2            5    19979739  6103    subway  Changi Airport Extension (CGL)   
3           21    33739391  6103    subway                Circle Line (CC)   
4          141   140918628  6103    subway      Circle Line Extension (CE)   
...        ...         ...   ...       ...                             ...   
1115      1309  1156550663  6103    subway                            None   
1116      1310  1156550664  6103    subway                            None   
1117      1311  1204951161  6101      rail                            None   
1118      1312  1221699926  6103    subway                            None   
1119      1313  1221699927  6103    subway                            None   

      layer bridge tunnel  Shape_Leng  \
0       1.0      T    

In [11]:
import geopandas as gpd
from itertools import combinations
from shapely.geometry import LineString
import pandas as pd

# 假设 cleaned_SUBWAY_gdf 和 final_gdf 已经加载到内存中
# cleaned_SUBWAY_gdf 包含多条地铁线路，geometry 列是 LineString 类型，并包含 'Shape_Leng' 列和 'stations_count' 列
# final_gdf 包含地块信息，geometry 列是 Polygon 或 MultiPolygon 类型

# 1. 筛选掉几何类型为 MultiLineString 的记录
cleaned_SUBWAY_gdf = cleaned_SUBWAY_gdf[cleaned_SUBWAY_gdf.geometry.apply(lambda geom: isinstance(geom, LineString))]

# 2. 计算每条地铁线路的站点数量 (假设每个点是一个站点)
cleaned_SUBWAY_gdf['stations_count'] = cleaned_SUBWAY_gdf.geometry.apply(lambda geom: len(geom.coords))

# 打印 cleaned_SUBWAY_gdf 的列名以便查看
print("cleaned_SUBWAY_gdf 列名:", cleaned_SUBWAY_gdf.columns.tolist())

# 3. 执行空间连接：找到与 cleaned_SUBWAY_gdf 中的 LINESTRING 相交的所有地块
intersected_parcels = gpd.sjoin(final_gdf, cleaned_SUBWAY_gdf, how="inner", predicate='intersects')

# 如果需要保存结果到文件
output_path = 'test.csv'
intersected_parcels.to_csv(output_path, index=False)
print(f"\n结果已保存至 {output_path}")

# 创建一个字典来存储结果三元组，键为 (parcel_a, parcel_b)，值为最小的距离和其他信息
results_dict = {}

# 4. 对每条地铁线路分别处理
for object_id, group in intersected_parcels.groupby('index_right'):  # 'index_right' 是根据 sjoin 自动生成的索引列，指向 cleaned_SUBWAY_gdf 的 OBJECTID
    # 获取当前线路的信息
    line_info = cleaned_SUBWAY_gdf.loc[object_id]
    shape_leng = line_info['Shape_Leng']
    stations_on_line = line_info.get('stations_count', None)
    
    if stations_on_line is None or stations_on_line == 0:
        print(f"线路 {object_id} 没有站点数量信息或站点数量为0，跳过此线路")
        continue
    
    # 如果该线路没有地块相交，则跳过
    if len(group) < 2:
        continue
    
    # 获取该线路上的所有地块
    parcels_on_line = group[['OBJECTID_1', 'geometry']].copy()  # 假设 'OBJECTID_1' 是地块的唯一标识符
    
    # 5. 计算地块之间的距离并找到最近的邻居
    for parcel_a, parcel_b in combinations(parcels_on_line.index, 2):
        # 计算两个地块在地铁线路上的顺序位置（假设等距分布）
        # 这里我们简化处理，直接用地块的索引作为顺序位置
        index_a = parcels_on_line.index.get_loc(parcel_a)
        index_b = parcels_on_line.index.get_loc(parcel_b)
        
        # 计算两个地块之间的站点数量差值
        station_diff = abs(index_a - index_b)
        
        # 计算地块之间的距离
        distance = station_diff * (shape_leng / stations_on_line)
        
        # 创建三元组 (地块一, 地块二, 距离, 线路信息)
        key = (parcel_a, parcel_b)
        if key not in results_dict or distance < results_dict[key]['distance']:
            results_dict[key] = {
                'parcel_a': parcel_a,
                'parcel_b': parcel_b,
                'distance': distance,
                **{col: line_info[col] for col in cleaned_SUBWAY_gdf.columns if col not in ['geometry']}
            }

# 将结果转换为 DataFrame 以便查看或进一步处理
results_df = pd.DataFrame(results_dict.values())

# 打印前几条记录以检查
print("\n三元组结果:")
print(results_df.head())

# 如果需要保存结果到文件
output_path = 'data/SUBWAY.csv'
results_df.to_csv(output_path, index=False)
print(f"\n结果已保存至 {output_path}")

cleaned_SUBWAY_gdf 列名: ['OBJECTID', 'osm_id', 'code', 'fclass', 'name', 'layer', 'bridge', 'tunnel', 'Shape_Leng', 'geometry', 'stations_count']

结果已保存至 test.csv

三元组结果:
   parcel_a  parcel_b  distance  OBJECTID     osm_id  code    fclass  \
0     16111     39181  0.000143       304  428907035  6105  monorail   
1     16111     39829  0.000286       304  428907035  6105  monorail   
2     16111     41695  0.000430       304  428907035  6105  monorail   
3     16111     43572  0.000573       304  428907035  6105  monorail   
4     16111     43887  0.000716       304  428907035  6105  monorail   

                name  layer bridge tunnel  Shape_Leng  stations_count  
0  Bukit Panjang LRT    1.0      T      F    0.025343             177  
1  Bukit Panjang LRT    1.0      T      F    0.025343             177  
2  Bukit Panjang LRT    1.0      T      F    0.025343             177  
3  Bukit Panjang LRT    1.0      T      F    0.025343             177  
4  Bukit Panjang LRT    1.0      T   

In [12]:
import geopandas as gpd
from itertools import combinations
from shapely.geometry import LineString
import pandas as pd

# 假设 cleaned_SUBWAY_gdf 和 final_gdf 已经加载到内存中
# cleaned_SUBWAY_gdf 包含多条地铁线路，geometry 列是 LineString 类型，并包含 'Shape_Leng' 列和 'stations_count' 列
# final_gdf 包含地块信息，geometry 列是 Polygon 或 MultiPolygon 类型

# 1. 筛选掉几何类型为 MultiLineString 的记录
cleaned_BUSWAY_gdf = cleaned_BUSWAY_gdf[cleaned_BUSWAY_gdf.geometry.apply(lambda geom: isinstance(geom, LineString))]

# 2. 计算每条地铁线路的站点数量 (假设每个点是一个站点)
cleaned_BUSWAY_gdf['stations_count'] = cleaned_BUSWAY_gdf.geometry.apply(lambda geom: len(geom.coords))

# 打印 cleaned_SUBWAY_gdf 的列名以便查看
print("cleaned_SUBWAY_gdf 列名:", cleaned_BUSWAY_gdf.columns.tolist())

# 3. 执行空间连接：找到与 cleaned_SUBWAY_gdf 中的 LINESTRING 相交的所有地块
intersected_parcels = gpd.sjoin(final_gdf, cleaned_BUSWAY_gdf, how="inner", predicate='intersects')

# 如果需要保存结果到文件
output_path = 'test.csv'
intersected_parcels.to_csv(output_path, index=False)
print(f"\n结果已保存至 {output_path}")

# 创建一个字典来存储结果三元组，键为 (parcel_a, parcel_b)，值为最小的距离和其他信息
results_dict = {}

# 4. 对每条地铁线路分别处理
for object_id, group in intersected_parcels.groupby('index_right'):  # 'index_right' 是根据 sjoin 自动生成的索引列，指向 cleaned_SUBWAY_gdf 的 OBJECTID
    # 获取当前线路的信息
    line_info = cleaned_BUSWAY_gdf.loc[object_id]
    shape_leng = line_info['Shape_Leng']
    stations_on_line = line_info.get('stations_count', None)
    
    if stations_on_line is None or stations_on_line == 0:
        print(f"线路 {object_id} 没有站点数量信息或站点数量为0，跳过此线路")
        continue
    
    # 如果该线路没有地块相交，则跳过
    if len(group) < 2:
        continue
    
    # 获取该线路上的所有地块
    parcels_on_line = group[['OBJECTID_1', 'geometry']].copy()  # 假设 'OBJECTID_1' 是地块的唯一标识符
    
    # 5. 计算地块之间的距离并找到最近的邻居
    for parcel_a, parcel_b in combinations(parcels_on_line.index, 2):
        # 计算两个地块在地铁线路上的顺序位置（假设等距分布）
        # 这里我们简化处理，直接用地块的索引作为顺序位置
        index_a = parcels_on_line.index.get_loc(parcel_a)
        index_b = parcels_on_line.index.get_loc(parcel_b)
        
        # 计算两个地块之间的站点数量差值
        station_diff = abs(index_a - index_b)
        
        # 计算地块之间的距离
        distance = station_diff * (shape_leng / stations_on_line)
        
        # 创建三元组 (地块一, 地块二, 距离, 线路信息)
        key = (parcel_a, parcel_b)
        if key not in results_dict or distance < results_dict[key]['distance']:
            results_dict[key] = {
                'parcel_a': parcel_a,
                'parcel_b': parcel_b,
                'distance': distance,
                **{col: line_info[col] for col in cleaned_BUSWAY_gdf.columns if col not in ['geometry']}
            }

# 将结果转换为 DataFrame 以便查看或进一步处理
results_df = pd.DataFrame(results_dict.values())

# 打印前几条记录以检查
print("\n三元组结果:")
print(results_df.head())

# 如果需要保存结果到文件
output_path = 'data/BUSWAY.csv'
results_df.to_csv(output_path, index=False)
print(f"\n结果已保存至 {output_path}")

cleaned_SUBWAY_gdf 列名: ['OBJECTID', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed', 'layer', 'bridge', 'tunnel', 'Shape_Leng', 'fclass字', 'fclass中', '描述', '分类', 'geometry', 'stations_count']

结果已保存至 test.csv

三元组结果:
   parcel_a  parcel_b  distance  OBJECTID  code   fclass  \
0     26009     63090  0.000062    154626  5141  service   
1     35388     36950  0.000177     23662  5141  service   
2     29804     75788  0.000181    142336  5153  footway   
3     92209     93978  0.000020    201763  5153  footway   
4      3361      9527  0.000336    145676  5153  footway   

                                         name   ref oneway  maxspeed  layer  \
0  105 Henderson Crescent Lobby Drive-Through  None      B         0    0.0   
1                           136 Joo Seng Road  None      B         0    0.0   
2                             24-hour Linkway  None      B         0    0.0   
3                                 Hougang B24  None      B         0    0.0   
4                    